# ガバナンス状態を検証する
データガバナンス機能で作成したデータやGakunin RDMで付与したメタデータを検証、実験の再現性を確認するタスクです。

## 検証する
ガバナンスシートに設定された検証項目に従って、ガバナンスの状態を検証します。<br>
ガバナンスシートが未登録の場合は[こちら](./governance_sheet.ipynb)からガバナンスシートを登録してください。<br>
検証にはメタデータの登録が必要です。メタデータ未登録の場合は[こちら](./metadata.ipynb)からメタデータを登録してください。<br>
<br>
ガバナンスシートの「再現性のレベル」が「設定しない」以外で登録されている場合、GovernedRunのトークン入力が必要になります。<br>
※入力したトークンの有効性が確認された場合はリサーチフローに記憶されますので次回以降は入力の必要はありません。


In [ ]:
# 検証する
import os
import json
import traceback

import panel as pn
from IPython.display import display, clear_output
from IPython.core.display import Javascript
from requests.exceptions import RequestException

from library.utils import dg_web
from library.task_director import TaskDirector
from library.utils.html.button import create_button
from library.utils.config import message as msg_config
from library.utils.widgets import MessageBox, Button, Alert
from library.utils.storage_provider import grdm
from library.utils.error import UnusableVault, ProjectNotExist, PermissionError
from library.utils.input import get_grdm_connection_parameters, get_goveredrun_token

notebook_name = 'validate.ipynb'

class Validator(TaskDirector):
    """ガバナンスの状態を検証するクラスです。
        
    
     Attributes:

        instance:
            _form_box (pn.WidgetBox): フォームを格納する。
            _msg_output (MessageBox): ユーザーに提示するメッセージを格納する。

    
    """

    def __init__(self, working_path:str) -> None:
        super().__init__(working_path, notebook_name)
        """jsonschemaから入力欄を生成し、データを取得するメソッドです。

        Args:
            working_path(str):実行Notebookファイルパス
        
        """

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

    def get_grdm_params(self):
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。
        
        Returns:
            token(str):GRDMのトークンの値を返す。
            project_id(str):プロジェクトIDの値を返す。
        
        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except PermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    def load_governedrun_token(self):
        """governed runのトークンを取得するメソッドです。
        
        Returns:
            token(str):governed runのトークンの値を返す。
        
        
        """
        token = ""
        try:
            token = get_goveredrun_token()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return token

    def get_data(self, token, project_id, remote_path):
        """GRDMの指定されたデータを取得するメソッドです。

        Args:
            token(str):トークンID
            project_id(str):プロジェクトID
            remote_path(Literal):リモート先のパス
        
        Returns:
            data(dict):GRDMの指定されたデータの値を返す。
        
        """
        data = {}
        try:
            data = grdm.download_json_file(
                token=token, api_url=grdm.API_V2_BASE_URL,
                project_id=project_id, remote_path=remote_path
            )
        except (FileNotFoundError, json.JSONDecodeError):
            # 既存データが無かった場合
            pass
        except Exception:
            message = msg_config.get('dg_web', 'get_data_error')
            self._msg_output.update_error(message)
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return data

    def alert_missing_file(self, path, msg):
        """必要なファイルが存在していない場合のメッセージ表示を作成するメソッドです。
        
        Args:
            path(Literal):ファイルが存在してるパス
            msg(str):メッセージ
        
        Returns:
            alert(Alert):メッセージを返す。
            button(pn.pane.HTML)メッセージと表示するボタンを返す。
        
        
        """
        alert = Alert.warning(msg)
        if not os.path.isfile(path):
            return [alert]
        button_width = 500
        obj = create_button(
            url=f'{path}?init_nb=true',
            target='_blank',
            msg=msg_config.get('dg_web', 'access_register'),
            button_width=f'{button_width}px'
        )
        button = pn.pane.HTML(obj, width=button_width)
        return [alert, button]

    @TaskDirector.task_cell("1")
    def generateFormScetion(self):
        """取得したデータを表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        try:
            # パラメータ取得
            self.grdm_token, self.project_id = self.get_grdm_params()
            self.govrun_token = None
            clear_output()

            # ガバナンスシートとメタデータの確認
            if self.grdm_token and not self._msg_output.has_message():
                govsheet = self.get_data(
                        token=self.grdm_token, project_id=self.project_id,
                        remote_path=dg_web.GOVSHEET_PATH
                    )
                metadata = self.get_data(
                        token=self.grdm_token, project_id=self.project_id,
                        remote_path=dg_web.METADATA_PATH
                    )
                if not govsheet:
                    path = './governance_sheet.ipynb'
                    msg = msg_config.get('dg_web', 'missing_govsheet')
                    self._form_box.extend(self.alert_missing_file(path, msg))
                elif not metadata:
                    path = './metadata.ipynb'
                    msg = msg_config.get('dg_web', 'missing_metadata')
                    self._form_box.extend(self.alert_missing_file(path, msg))
                else:
                    # 再現性検証の準備
                    if dg_web.need_govrun_token(govsheet, metadata):
                        self.govrun_token = self.load_governedrun_token()

            # ボタンの表示
            if not self._msg_output.has_message() and not self._form_box.objects:
                self.submit_button_title = msg_config.get('dg_web', 'validate')
                self.submit_button = Button(width=500)
                self.submit_button.set_looks_init(self.submit_button_title)
                self.submit_button.on_click(self.submit)
                self._form_box.append(self.submit_button)

        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    @TaskDirector.callback_form('検証する')
    def submit(self, event):
        """検証し結果を表示する
        
                ボタンが押された時、dg_webに接続して検証し、その検証結果を表示させるようにする。

        
        """
        self.submit_button.set_looks_processing(msg_config.get('dg_web', 'validating'))
        try:
            result = dg_web.validate(
                scheme=dg_web.SCHEME, domain=dg_web.DOMAIN,
                grdm_token=self.grdm_token, project_id=self.project_id, govrun_token=self.govrun_token
            )
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
            return
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.log.error(message)
            self._msg_output.update_error(message)
            self.submit_button.set_looks_init(self.submit_button_title)
            return
        self._form_box.clear()
        title = '# ' + msg_config.get('dg_web', 'validation_result')
        self._form_box.append(pn.pane.Markdown(title))
        result = json.dumps(result, ensure_ascii=False, indent=4)
        styles = {
            'background': '#f5f5f5',
            'border': '1px solid black',
            'padding': '10px'
        }
        self._form_box.append(pn.pane.Str(result, styles=styles))
        # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
        self.done_task()

Validator(os.path.abspath('__file__')).generateFormScetion()

## 検証結果を確認する

検証結果の一覧を表示します。<br>
再現性検証が実施されている場合、検証実施直後には検証結果が得られない可能性があります。しばらく時間を置いてから再度確認してください。<br>

In [ ]:
# 検証結果を確認する
import os
import traceback
import json
import panel as pn
import pandas as pd

from requests.exceptions import RequestException
from IPython.core.display import Javascript
from IPython.display import display, clear_output

from library.utils import dg_web
from library.utils.error import (UnauthorizedError, ProjectNotExist,
                                 UnusableVault, NotFoundContentsError, PermissionError)
from library.task_director import TaskDirector
from library.utils.input import get_grdm_connection_parameters
from library.utils.widgets import MessageBox
from library.utils.config import message as msg_config

notebook_name = 'validate.ipynb'

class ValidationResult(TaskDirector):
    """検証結果の一覧の結果を表示するクラスです。


    Attributes:
    
        instance:
            _form_box (pn.WidgetBox): フォームを格納する。
            _msg_output (MessageBox): ユーザーに提示するメッセージを格納する。

    
    """


    def __init__(self, working_path: str) -> None:
        """ValidationResult コンストラクタのメソッドです

        Args:
            working_path(str):実行Notebookファイルパス
        
        """
        super().__init__(working_path, notebook_name)
        

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900

        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

    def get_grdm_params(self):
        """GRDMのトークンとプロジェクトIDを取得するメソッドです。
        
        Returns:
            token(str):GRDMのトークンの値を返す。
            project_id(str):プロジェクトIDの値を返す。

        """
        token = ""
        project_id = ""
        try:
            token, project_id = get_grdm_connection_parameters()
        except UnusableVault as e:
            message = msg_config.get('form', 'no_vault')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except PermissionError:
            message = msg_config.get('form', 'insufficient_permission')
            self._msg_output.update_error(message)
            self.log.error(traceback.format_exc())
        except ProjectNotExist as e:
            self._msg_output.update_error(str(e))
            self.log.error(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)
        return token, project_id

    @TaskDirector.task_cell("2")
    def generateFormScetion(self):
        """取得したデータを表示するメソッドです。
            検証結果を表示するボタンを表示させ、そのボタンを押すことでデータが表示されるようにする。
        
        """
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()
        # フォーム定義
        try:
            self.token, self.project_id = self.get_grdm_params()
            content = self.get_validate()
            if content:
                self.display_table(content)
            clear_output()
            # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
            self.done_task()
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

        # フォーム表示
        pn.extension("tabulator")
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    def get_validate(self) -> dict:
        """検証結果一覧を取得するメソッドです。

            DG_Webに接続し、スキーマ、ドメイン名、トークン、プロジェクトIDを用いて検証結果の一覧を取得する。
        
        Returns:
            content(dict):検証結果一覧のデータを返す。

        
        
        """
        content = {}
        try:
            content = dg_web.get_validations(dg_web.SCHEME, dg_web.DOMAIN, self.token, self.project_id)
        except NotFoundContentsError:
            content = {}
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return content

    def display_table(self, content: dict):
        """検証結果一覧を表示するメソッドです。

        Args:
            content(dict):検証結果のデータに格納されているキーと値
        
        
        """
        self.id_list, status_list = self.get_table(content)

        df = pd.DataFrame({
            msg_config.get('dg_web', 'validate_id'): self.id_list,
            msg_config.get('dg_web', 'status'): status_list
        })

        select_table = pn.widgets.Tabulator(
            df,
            show_index=False,
            header_align='center',
            configuration={'columnDefaults': {'headerSort': False}},
            editors={msg_config.get('dg_web', 'validate_id'): None, msg_config.get('dg_web', 'status'): None}
        )
        select_table.style.map(self.text_color)
        select_table.on_click(self.click)

        self._form_box.append(select_table)

    def get_validation_result(self, validation_id):
        """指定した検証結果を取得するメソッドです。
            DG_Webに接続し、スキーマ、ドメイン名、トークン、プロジェクトID、検証IDを用いて指定した検証結果を取得する。

        Args:
            validation_id(list):検証ID

        Returns:
            result(dict):検証結果のデータを返す。

        
        """
        result = {}
        try:
            result = dg_web.get_validations_validationId(dg_web.SCHEME, dg_web.DOMAIN, self.token, self.project_id, validation_id)
        except NotFoundContentsError:
            result = {}
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self._msg_output.update_warning(message)
            self.log.warning(traceback.format_exc())
        except RequestException as e:
            message = msg_config.get('DEFAULT', 'connection_error')
            self._msg_output.update_error(f'{message}\n{str(e)}')
            self.log.error(f'{message}\n{traceback.format_exc()}')
        return result

    @TaskDirector.callback_form('検証結果詳細を表示する')
    def click(self, event):
        """検証結果詳細を表示するメソッドです。
            検証結果のデータを取得後、検証IDを用いて検証結果を表示する。
        
        """
        try:
            validation_id = self.id_list[event.row]
            validation_result = self.get_validation_result(validation_id)
            validation_detail = json.dumps(validation_result, ensure_ascii=False, indent=4)

            self._msg_output.clear()
            title = '# ' + msg_config.get('dg_web', 'validation_result')
            self._msg_output.append(pn.pane.Markdown(title))
            styles = {
                'background': '#f5f5f5',
                'border': '1px solid black',
                'padding': '10px'
            }
            self._msg_output.append(pn.pane.Str(validation_detail, styles=styles))

        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self._msg_output.update_error(message)
            self.log.error(message)

    def text_color(self, text):
        """表のテキストの色を変更するメソッドです。
        
        Args:
            text(str):テキストの色
        
        Returns:
            color(str):条件に合った色を返す。
        
        """
        color = ''
        if text == 'SUCCESS':
            color = 'green'
        elif text == 'FAILURE':
            color = 'red'
        elif text == 'RUNNING':
            color = 'purple'
        elif text == 'ERROR':
            color = 'orange'
        else:
            color = 'black'
        return 'color: %s' % color

    def get_table(self, content):
        """一覧に表示させる要素を取得するメソッドです。
            contentのkeyが'runs'でない場合、空のリストを返す
            ID、statusの要素が無かった場合、処理をスキップする

        Args:
            content(dict):検証結果のデータ

        Returns:
            id_list(list):検証結果のデータのIDのリスト
            status_list(list):検証結果のステータスのリスト

            
        """
        id_list = []
        status_list = []

        if 'runs' not in content:
            return id_list, status_list

        for table in content.get('runs'):
            if 'id' not in table or 'status' not in table:
                continue
            else:
                id_list.append(table.get('id'))
                status_list.append(table.get('status').upper())
        return id_list, status_list

ValidationResult(working_path=os.path.abspath('__file__')).generateFormScetion()

## サブフローメニューを表示する

サブフローメニューへ遷移するボタンを表示します。

In [ ]:
"""
サブフローメニューに戻るためにサブフローメニューのページに飛ぶための画面とボタンを用意する。
"""
# サブフローメニューを表示する
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector

script_file_name = "validate"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()